## A Visual History of Interpretation in Image Recognition

This notebook reproduces the history-of-interpretation [blog post](https://gradio.app/blog/interpretation-history), by the [Gradio](https://github.com/gradio-app/gradio) team. We relied heavily on [PAIR-code's implementation](https://github.com/PAIR-code/saliency) of the papers. 

Find the colab version [here](https://colab.research.google.com/drive/1IxhImCFknNMctIonSo98nkco2ufKmfdj?usp=sharing). 

### Imports and Setup

In [2]:
!pip install tf-slim gradio wget -q
import tensorflow.compat.v1 as tf
import tf_slim as slim
import sys
import inception_v3
import saliency
import requests
import gradio as gr
import wget
import tarfile
import numpy as np

import os
if not os.path.exists('inception_v3.ckpt'):
    wget.download("http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz")
    tar = tarfile.open("inception_v3_2016_08_28.tar.gz")
    tar.extractall()
    tar.close()

You should consider upgrading via the '/Users/aliabdalla/anaconda3/bin/python -m pip install --upgrade pip' command.


### Setting up the graph, and adding a logit tensor

In [ ]:
ckpt_file = './inception_v3.ckpt'
graph = tf.Graph()

with graph.as_default():
    images = tf.placeholder(tf.float32, shape=(None, 299, 299, 3))

    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        _, end_points = inception_v3.inception_v3(images, is_training=False, num_classes=1001)

        # Restore the checkpoint
        sess = tf.Session(graph=graph)
        saver = tf.train.Saver()
        saver.restore(sess, ckpt_file)

    # Construct the scalar neuron tensor.
    logits = graph.get_tensor_by_name('InceptionV3/Logits/SpatialSqueeze:0')
    neuron_selector = tf.placeholder(tf.int32)
    y = logits[0][neuron_selector]

    # Construct tensor for predictions.
    prediction = tf.argmax(logits, 1)

### Initializing and creating the different saliency methods

In [ ]:
gradients = saliency.GradientSaliency(graph, sess, y, images)
guided = saliency.GuidedBackprop(graph, sess, y, images)
integrated = saliency.IntegratedGradients(graph, sess, y, images)
blur_ig = saliency.BlurIG(graph, sess, y, images)

In [24]:
def vanilla_gradients(image):
    image = image / 127.5 - 1.0
    prediction_class = sess.run(prediction, feed_dict = {images: [image]})[0]
    vanilla_mask_3d = gradients.GetMask(image, feed_dict = {neuron_selector: prediction_class})
    vanilla_mask_grayscale = saliency.VisualizeImageGrayscale(vanilla_mask_3d)
    return vanilla_mask_grayscale.tolist()

def smoothgrad(image):
    image = image / 127.5 - 1.0
    prediction_class = sess.run(prediction, feed_dict = {images: [image]})[0]
    smoothgrad_mask_3d = gradients.GetSmoothedMask(image, feed_dict = {neuron_selector: prediction_class})
    smoothgrad_mask_grayscale = saliency.VisualizeImageGrayscale(smoothgrad_mask_3d)
    return smoothgrad_mask_grayscale.tolist()

def guided_backprop(image):
    image = image / 127.5 - 1.0
    prediction_class = sess.run(prediction, feed_dict = {images: [image]})[0]
    vanilla_guided_backprop_mask_3d = guided.GetMask(
    image, feed_dict = {neuron_selector: prediction_class})
    vanilla_mask_grayscale = saliency.VisualizeImageGrayscale(vanilla_guided_backprop_mask_3d)
    return vanilla_mask_grayscale.tolist()

def integrated_smoothgrad(image):
    image = image / 127.5 - 1.0
    prediction_class = sess.run(prediction, feed_dict = {images: [image]})[0]
    baseline = np.zeros(image.shape)
    baseline.fill(-1)
    smoothgrad_integrated_gradients_mask_3d = integrated.GetSmoothedMask(
    image, feed_dict = {neuron_selector: prediction_class}, x_steps=25, x_baseline=baseline)
    smoothgrad_mask_grayscale = saliency.VisualizeImageGrayscale(smoothgrad_integrated_gradients_mask_3d)
    return smoothgrad_mask_grayscale.tolist()

def blur_IG_vanilla(image):
    image = image / 127.5 - 1.0
    prediction_class = sess.run(prediction, feed_dict = {images: [image]})[0]
    baseline = np.zeros(image.shape)
    baseline.fill(-1)

    blur_ig_mask_3d = blur_ig.GetMask(
    image, feed_dict = {neuron_selector: prediction_class})

    blur_ig_mask_grayscale = saliency.VisualizeImageGrayscale(blur_ig_mask_3d)
    return blur_ig_mask_grayscale.tolist()

### Setting up classifier and Gradio

In [12]:
# Download human-readable labels for ImageNet.
inception_net = tf.keras.applications.InceptionV3() # load the model

# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")


def classify_image(inp):
    inp = inp.reshape((-1, 299, 299, 3))
    inp = tf.keras.applications.inception_v3.preprocess_input(inp)
    prediction = inception_net.predict(inp).flatten()
    return {labels[i]: float(prediction[i]) for i in range(1000)}

image = gr.inputs.Image(shape=(299, 299, 3))
label = gr.outputs.Label(num_top_classes=3)

examples = [["doberman.png"], ["dog.png"]]

gr.Interface(classify_image, image, label, capture_session=True, examples=examples,
             allow_flagging=False).launch()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
96116736/96112376 [==============================] - 2s 0us/step


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://42271.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://42271.gradio.app')

### Leave-One-Out
Default out of the box interpretation in Gradio. 

In [15]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation="default", examples=examples,
             allow_flagging=False).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://10479.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7863/',
 'https://10479.gradio.app')

### Vanilla Gradient Ascent [2009 and 2013]

Paper: [Visualizing Higher-Layer Features of a Deep Network (2009)](https://www.researchgate.net/profile/Aaron_Courville/publication/265022827_Visualizing_Higher-Layer_Features_of_a_Deep_Network/links/53ff82b00cf24c81027da530.pdf)

Paper: [Visualizing Image Classification Models and Saliency Maps (2013)](https://arxiv.org/abs/1312.6034)




In [16]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation=vanilla_gradients, examples=examples,
             allow_flagging=False).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://16147.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7864/',
 'https://16147.gradio.app')

### Guided Back-Propogation [2014]

Paper: [Striving for Simplicity: The All Convolutional Net (2014)](https://arxiv.org/abs/1412.6806)



In [17]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation=guided_backprop, examples=examples,
             allow_flagging=False).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://41416.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7865/',
 'https://41416.gradio.app')

### SmoothGrad [2017]

Paper: [SmoothGrad: removing noise by adding noise (2017)](https://arxiv.org/abs/1706.03825)



In [18]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation=smoothgrad, examples=examples,
             allow_flagging=False).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://35400.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7866/',
 'https://35400.gradio.app')

### Integrated Gradients [2017]

Paper: [Axiomatic Attribution for Deep Networks (2017)](https://arxiv.org/abs/1703.01365)

**Note**: This method is *very* slow


In [ ]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation=integrated_smoothgrad, examples=examples,
             allow_flagging=False).launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://39815.gradio.app
Interface loading below...


### Blur Integrated Gradients [2020]

Paper: [Attribution in Scale and Space (2020)](https://arxiv.org/pdf/2004.03383)

**Note**: This method is *very* slow



In [26]:
gr.Interface(classify_image, image, label, capture_session=True, interpretation=blur_IG_vanilla, examples=examples,
             allow_flagging=False).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://21482.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7871/',
 'https://21482.gradio.app')